In [1]:
!pip install -q cassio datasets langchain openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.8/258.8 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [2]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
#from openai import OpenAI
from langchain.embeddings import OpenAIEmbeddings

from datasets import load_dataset

import cassio

In [3]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.3 MB/s eta 0:00:00


In [4]:
from PyPDF2 import PdfReader

In [5]:
ASTRA_DB_APPLICATION_TOKEN=""
ASTRA_DB_ID=""
OPENAI_API_KEY="OPENAI_API_KEY"

In [6]:
pdfreader=PdfReader('path.pdf')

In [7]:
from typing_extensions import Concatenate
raw_text=''
for i,page in enumerate(pdfreader.pages):
  content = page.extract_text()
  if content:
    raw_text+=content

In [8]:
print(raw_text)

CASE REPORT
Archives of Clinical and Experimental Surgery
2022 VOL 11, NO. 8, PAGES 01-08
Open Access
Comprehensive Treatment of a Rare Primary Malignant Pericardial Mesothelioma: A 
Case Report and Literature Review
Chunyu Liu, Bin Zhao, Weiwei Cheng, Dongliang Ma, Xing Wei, Shunye Zhang
*
Department of Cardiac Surgery, Shanxi Cardiovascular Hospital, Taiyuan of Shanxi province, China
Contact:
 Zhang S, Tel: +8613663618362, E-mail: xxgwk@139.com
Copyright: 
© 2022 The Authors. This is an open access article under the terms of the Creative Commons Attribution NonCommercial ShareAlike 4.0   
(https://creativecommons.org/licenses/by-nc-sa/4.0/).
Introduction
Cardiac tumours are rare [1]. Pericardial malignan -
cies account for only 0.0025% of all malignant tu -
mours and less than 2% of all cardiac tumours [2,3]. 
PMPM, originating from pericardial mesothelial cells, 
can be divided into diffuse and localised types. The 
diffuse type is characterised by obvious bleeding, 
exudation, and 

connection to database

In [9]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

ERROR:cassandra.connection:Closing connection <AsyncoreConnection(139190433683280) 78459230-d954-46e9-9f6a-07d9a7362e61-us-east1.db.astra.datastax.com:29042:e1b20a5d-8e92-4b5a-be5a-584a278235b5> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


LangChain Embedding or LLM object

In [13]:
llm= OpenAI(openai_api_key='OPENAI_API_KEY')


In [14]:
embedding = OpenAIEmbeddings(openai_api_key='OPENAI_API_KEY')

In [15]:
astra_vector_store = Cassandra(
    embedding=embedding,
    table_name='qa_mini_demo',
    session=None,
    keyspace=None,
)

In [16]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=800,
    chunk_overlap=200,
    length_function=len,
)

texts = text_splitter.split_text(raw_text)

In [17]:
texts[:50]

['CASE REPORT\nArchives of Clinical and Experimental Surgery\n2022 VOL 11, NO. 8, PAGES 01-08\nOpen Access\nComprehensive Treatment of a Rare Primary Malignant Pericardial Mesothelioma: A \nCase Report and Literature Review\nChunyu Liu, Bin Zhao, Weiwei Cheng, Dongliang Ma, Xing Wei, Shunye Zhang\n*\nDepartment of Cardiac Surgery, Shanxi Cardiovascular Hospital, Taiyuan of Shanxi province, China\nContact:\n Zhang S, Tel: +8613663618362, E-mail: xxgwk@139.com\nCopyright: \n© 2022 The Authors. This is an open access article under the terms of the Creative Commons Attribution NonCommercial ShareAlike 4.0   \n(https://creativecommons.org/licenses/by-nc-sa/4.0/).\nIntroduction\nCardiac tumours are rare [1]. Pericardial malignan -\ncies account for only 0.0025% of all malignant tu -',
 '(https://creativecommons.org/licenses/by-nc-sa/4.0/).\nIntroduction\nCardiac tumours are rare [1]. Pericardial malignan -\ncies account for only 0.0025% of all malignant tu -\nmours and less than 2% of all ca

In [18]:
astra_vector_store.add_texts(texts[:50])
print("Inserted %i headlines." %len(texts[:50]))
astra_vector_index =VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 39 headlines.


Q/A session start

In [19]:
f_q=True
while True:
  if f_q:
    query_text=input("\nEnter your Question [or type quit to exit]:").strip()
  else:
    query_text =input("\n What is your Next Question:").strip()

  if query_text.lower()=="quit":
    break

  if query_text=="":
    continue

  f_q=False

  print("\nQuestion:\"%s\""%query_text)
  answer=astra_vector_index.query(query_text, llm=llm).strip()
  print("answer:\"%s\"\n"%answer)

  print("first doc by relevence:")
  for doc,score in astra_vector_store.similarity_search_with_score(query_text, k=4):
    print(" [%0.4f]\"%s...\""%(score,doc.page_content[:84]))



Enter your Question [or type quit to exit]:provide the table provided in document

Question:"provide the table provided in document"


answer:"I'm sorry, I don't see a table provided in the document. Can you please specify which table you are referring to?"

first doc by relevence:


 [0.8675]"Ethics approval and consent to participate
Not applicable.
Consent for publication
T..."
 [0.8640]"5 Arch Clin Exp Surg • 2022 • Vol 11 • Issue 8
Figure 6.
Figure 7.  Medical evidence..."
 [0.8638]"Envision) - Third year students  
4. Organisers - Fourth year s tudents  
 
Descript..."
 [0.8638]"Envision) - Third year students  
4. Organisers - Fourth year s tudents  
 
Descript..."

 What is your Next Question:quit
